In [8]:
import pandas as pd
import pickle

In [5]:
class formesPatron:
    '''
    Accumulateur de formes correspondant à un patron pour calcul de la Généralisation Minimale (cf. MGL)
    '''
    def __init__(self):
        self.formes=[]

#    def __repr__(self):
#        return ','.join(self.calculerGM())
        
    def ajouterForme(self,forme):
        self.formes.append(forme)
        
    def calculerGM(self):
        minLongueur=len(min(self.formes, key=len))
        maxLongueur=len(max(self.formes, key=len))
        if debug: 
            # print (minLongueur, maxLongueur, file=logfile)
            print (minLongueur, maxLongueur)
        positions=[]
        if maxLongueur>minLongueur:
            positions.append("*")
        for i in xrange(minLongueur, 0, -1):
            phonemes=set([x[-i] for x in self.formes])
            if debug: 
                # print (phonemes, file=logfile)
                print (phonemes)
            if "." in phonemes:
                positions.append(".")
            else:
                positions.append("".join(fs.lattice[phonemes].extent))
        return patron2regexp(positions)

class pairePatrons:
    '''
    Accumulateur de triplets (f1,f2,patron) correspondant à une paire pour calcul des Généralisations Minimales (cf. MGL)
    '''
    def __init__(self,case1,case2):
        self.patrons1={}
        self.patrons2={}
        self.case1=case1
        self.case2=case2

#    def __repr__(self):
#        return ','.join(self.calculerGM())
        
    def ajouterFormes(self,forme1,forme2,patron):
#        print (forme1,forme2,patron, file=logfile)
        patron12=patron
        (pat1,pat2)=patron.split("-")
        patron21=pat2+"-"+pat1
#        print (patron12,patron21, file=logfile)
        if not patron12 in self.patrons1:
            self.patrons1[patron12]=formesPatron()
        self.patrons1[patron12].ajouterForme(forme1)
        if not patron21 in self.patrons2:
            self.patrons2[patron21]=formesPatron()
        self.patrons2[patron21].ajouterForme(forme2)
        
        
    def calculerGM(self):
        resultat1={}
        for patron in self.patrons1:
            if debug: 
                # print ("patron1", patron, file=logfile)
                print ("patron1", patron)
            resultat1[patron]=self.patrons1[patron].calculerGM()
        resultat2={}
        for patron in self.patrons2:
            if debug: 
                # print ("patron2", patron, file=logfile)
                print ("patron2", patron)
            resultat2[patron]=self.patrons2[patron].calculerGM()
        return (resultat1,resultat2) 

In [6]:
class paireClasses:
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classes1=classesPaire(case1,case2)
        self.classes2=classesPaire(case2,case1)

    def ajouterPatron(self,n,patron,motif):
        if n==1:
            self.classes1.ajouterPatron(patron,motif)
        elif n==2:
            self.classes2.ajouterPatron(patron,motif)
        else:
            if debug: 
                # print ("le numéro de forme n'est pas dans [1,2]",n, file=logfile)
                print ("le numéro de forme n'est pas dans [1,2]",n)

    def ajouterPaire(self,forme1,forme2):
        self.classes1.ajouterPaire(forme1,forme2)
        self.classes2.ajouterPaire(forme2,forme1)
        
    def calculerClasses(self):
        return(self.classes1,self.classes2)

    
class classesPaire:
    '''
    Gestion des patrons, des classes et des transformations
    
    ajouterPatron : ajoute un patron et son motif associé (MGL)
    ajouterPaire : ajoute une paire de formes, calcule la classe de la forme1 et la règle sélectionnée
    sortirForme : cacule les formes de sortie correspondant à la forme1 avec leurs coefficients respectifs
    '''
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classe={}
        self.nbClasse={}
        self.patrons={}
        self.entree={}
        self.sortie={}
        self.classeCF={}
        self.nbClasseCF={}
    
    def ajouterPatron(self,patron,motif):
        self.patrons[patron]=motif
        (entree,sortie)=patron.split("-")
        self.entree[patron]=entree.replace(u".",u"(.)")
        self.sortie[patron]=remplacementSortie(sortie)
    
    def ajouterPaire(self,forme1,forme2):
        '''
        on calcule la classe de la paire idClasseForme et la règle sélectionnée
        on incrémente le compteur de la classe et celui de la règle sélectionnée à l'intérieur de la classe
        '''
        classeFormeCF=[]
        regleFormeCF=""
        classeForme=[]
        regleForme=""
        for patron in self.patrons:
            filterF1=".*"+patron.split("-")[0]+"$"
            if re.match(filterF1,forme1):
                classeFormeCF.append(patron)
                if forme2==re.sub(self.entree[patron]+"$",self.sortie[patron],forme1):
                    regleFormeCF=patron
            filterF1=self.patrons[patron]
            if re.match(filterF1,forme1):
                classeForme.append(patron)
                '''
                le +"$" permet de forcer l'alignement à droite pour les transformations suffixales
                '''
                if forme2==re.sub(self.entree[patron]+"$",self.sortie[patron],forme1):
                    regleForme=patron
        idClasseFormeCF=", ".join(classeFormeCF)
        if not idClasseFormeCF in self.classeCF:
            self.classeCF[idClasseFormeCF]={}
            self.nbClasseCF[idClasseFormeCF]=0
        if not regleFormeCF in self.classeCF[idClasseFormeCF]:
            self.classeCF[idClasseFormeCF][regleFormeCF]=0
        self.nbClasseCF[idClasseFormeCF]+=1
        self.classeCF[idClasseFormeCF][regleFormeCF]+=1
        
        idClasseForme=", ".join(classeForme)
        if not idClasseForme in self.classe:
            self.classe[idClasseForme]={}
            self.nbClasse[idClasseForme]=0
        if not regleForme in self.classe[idClasseForme]:
            self.classe[idClasseForme][regleForme]=0
        self.nbClasse[idClasseForme]+=1
        self.classe[idClasseForme][regleForme]+=1

    def sortirForme(self,forme,contextFree=False):
        classeForme=[]
        sortieForme={}
        for patron in self.patrons:
            if contextFree:
                filterF1=".*"+patron.split("-")[0]+"$"
            else:
                filterF1=self.patrons[patron]
            if re.match(filterF1,forme):
                classeForme.append(patron)
        if classeForme:
            idClasseForme=", ".join(classeForme)
            if contextFree:
                nbClasse=self.nbClasseCF
                classe=self.classeCF
            else:
                nbClasse=self.nbClasse
                classe=self.classe
            if idClasseForme in nbClasse:
                nTotal=nbClasse[idClasseForme]
                for patron in classe[idClasseForme]:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(classe[idClasseForme][patron])/nTotal
            else:
                if debug: 
                    # print (forme, file=logfile)
                    # print ("pas de classe",idClasseForme, file=logfile)
                    # print ("%.2f par forme de sortie" % (float(1)/len(classeForme)), file=logfile)
                    print (forme)
                    print ("pas de classe",idClasseForme)
                    print ("%.2f par forme de sortie" % (float(1)/len(classeForme)))
                nTotal=len(classeForme)
                for patron in classeForme:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(1)/nTotal
        else:
            if debug:
                # print (forme, file=logfile) 
                # print ("pas de patron", file=logfile)
                print (forme) 
                print ("pas de patron")
        return sortieForme
        

In [13]:
with open("/Users/gilles/ownCloud/Recherche/Boye/HDR/Data/L4L/IMM21/IMM21-01-X-Regles.pkl") as inFile:
    reglesL4L=pickle.load(inFile)
# reglesL4L[(u'ps2P', u'is1S')].patrons

In [14]:
with open("/Users/gilles/ownCloud/Recherche/Boye/HDR/Data/L4L/IMM21/IMM21-00-X-Regles.pkl") as inFile:
    reglesHead=pickle.load(inFile)
# reglesHead[(u'ps2P', u'is1S')].patrons

In [15]:
with open("/Users/gilles/ownCloud/Recherche/Boye/HDR/Data/L4L/IMM21/IMM21-02-X-Regles.pkl") as inFile:
    reglesTail=pickle.load(inFile)
# reglesTail[(u'ps2P', u'is1S')].patrons

In [36]:
with open("/Users/gilles/ownCloud/Recherche/Boye/HDR/Data/L4L/IMM21/IMM21-01-X-Morphomes-Regles.pkl") as inFile:
    reglesMorphomes=pickle.load(inFile)
# reglesTail[(u'ps2P', u'is1S')].patrons

In [35]:
paire=(u'ppFS', u'inf')
nRegles="L4L Head Tail".split(" ")
for n,regles in enumerate([reglesL4L,reglesHead,reglesTail]):
    print nRegles[n],regles[paire].nom
    print
    for k,v in regles[paire].patrons.iteritems():
        print k,v
    print
    print "================="
    print

L4L ppFS-inf

e-Etr ^(.*)ne$
Ert-rir ^(.*[uOo][fv])Ert$
y-avwar ^(.*)y$
iz-âdr ^(.*)priz$
zy-dr ^(.*)kuzy$
y-Etr ^(.*[pk][E96aO][mnr])y$
O.t-u.ir ^mOrt$
iz-Erir ^(.*[E96aOêûâô])kiz$
- ^(.*[ptkbdgfsSvzZmnJNjlrwHiyEe926auOoêûâô][ptkbdgfsSvzZmnJNjlrwHE96aOêûâô])e$
oz-Or ^(.*)kloz$
Eky-ivr ^vEky$
iz-war ^(.*)asiz$
-r ^(.*[ptkbdgfsSvzZmnJNjlrwHiyEe926auOoêûâô][iye2])$
y-ir ^(.*[ptbdfsvzmnlr])y$
ly-dr ^muly$
t-dr ^(.*[ptkbdgfsSvzZjlrwH][iyEe926uOoêûô])t$
y-uvwar ^(.*[E96O])my$
y-war ^(.*[fsSvzZlr])y$
t-r ^(.*[ptkbdgfsSvzZjrwH][iEe])t$
y-r ^(.*[ptbdfsvzrE96O][jrwHiyEe926auOoêûâô][ptkbdg])y$
y-6vwar ^(.*[tdsz])y$
y-watr ^akry$
i-r ^(.*)sHivi$
z-r ^sirkôsiz$
iz-Etr ^(.*)miz$


Head ppFS-inf

e-Etr ^(.*)ne$
Ert-rir ^(.*[uOo][fv])Ert$
y-avwar ^y$
iz-âdr ^(.*[rE6a])priz$
zy-dr ^(.*)kuzy$
y-Etr ^(.*[pk][E96aO][mnr])y$
iz-Erir ^(.*[E96aOêûâô])kiz$
- ^(.*[ptkbdgfsSvzZmnJNjlrwHiyEe926auOoêûâô][ptkbdgfsSvzZmnJNjlrwHE96Oêûô])e$
oz-Or ^kloz$
Eky-ivr ^vEky$
iz-war ^asiz$
-r ^(.*[ptkbdgfsSvzZmnJNjlrwHE96aO

In [42]:
paire=(u'fi3S', u'ii3S')
for k,v in reglesL4L[paire].patrons.iteritems():
    print k,v
    print k,reglesMorphomes[paire].patrons[k]
    print


ra-sE ^(.*[ptkbdgfsSvzZmnJNjlrwHE96aOêûâô])ira$
ra-sE ^(.*[ptkbdgfsSvzZmnJNjlrwHiyEe926auOoêûâô])ira$

9.6ra-6.E ^(.*)dEZ9n6ra$
9.6ra-6.E ^(.*)dEZ9n6ra$

jEra-EjE ^(.*)sjEra$
jEra-EjE ^(.*)sjEra$

ira-E ^(.*[ptkbdgfsSvzZmnJNjrwHiyEe926auOoêûâô][fvjrwHiyEe926auOoêûâô][ptkbdgfsSvzZmnJNjr])ira$
ira-E ^(.*[ptkbdgfsSvzZmnJNjrwHiyEe926auOoêûâô][fvjrwHiyEe926auOoêûâô][ptkbdgfsSvzZmnJNjr])ira$

ra-wE ^(.*[ptkfsS][lr])ura$
ra-wE ^(.*[ptkbdgfsSvzZ][lr])ura$

Odra-alE ^(.*[fv])Odra$
Odra-alE ^(.*[fv])Odra$

6r-E ^sâbl6r$
6r-E ^sâbl6r$

d6ra- ^sud6ra$
d6ra- ^sud6ra$

ra-E ^(.*[ptkbdgfsSvzZmnJNjlrwHE96Oêûô][mnJNjlrwHiyEe926auOoêûâô][ptkbdgfsSvzZjrE6])ra$
ra-E ^(.*[ptkbdgfsSvzZmnJNjlrwHE96Oêûô][mnJNjlrwHiyEe926auOoêûâô][ptkbdgfsSvzZjrE6])ra$

O.6ra-o.E ^(.*)O([ptbdfsvzmnlr])6ra$
O.6ra-o.E ^(.*)O([ptbdfsvzmnlr])6ra$

Ora-avE ^sOra$
Ora-avE ^sOra$

9.6ra-2.E ^(.*[ptkbdgfsSvzZmnJNlr])9([ptkbdgfsSvzZmnJNjlr])6ra$
9.6ra-2.E ^(.*[ptkbdgfsSvzZmnJNjlr])9([ptkbdgfsSvzZmnJNjlr])6ra$

ra-zE ^(.*[ptbdfsvzjlH][i

In [32]:
morphomes=pd.read_pickle("/Users/gilles/ownCloud/Recherche/Boye/HDR/Data/L4L/IMM21/IMM21-01-X-Morphomes.pkl")
print len(morphomes["case morphome".split(" ")].drop_duplicates())
morphomes["case morphome".split(" ")].drop_duplicates()

32


,case,morphome
0,inf,inf
1,ppMS,ppMP/ppMS
2,ppFS,ppFS/ppFP
3,pi3S,pi2S/pi3S
4,pP,pP
6,pi1S,pi1S
7,pi3P,pi3P
8,pi2P,pi2P
11,ai3S,ai3S/ai2S/is3S
12,fi3S,fi2S/fi3S


In [46]:
reglesL4L[(u'fi3S', u'ii3S')].patrons

{u'6r-E': u'^s\xe2bl6r$',
 u'6ra-E': u'^(.*[ptkbdgfsSvzZmnJNjlrwHiyEe926auOo\xea\xfb\xe2\xf4][ptkbdgfsSvzZmnJNjlr])6ra$',
 u'9.6ra-2.E': u'^(.*[ptkbdgfsSvzZmnJNlr])9([ptkbdgfsSvzZmnJNjlr])6ra$',
 u'9.6ra-6.E': u'^(.*)dEZ9n6ra$',
 u'E.6ra-6.E': u'^(.*[ptkbdgfsSvzZmnJNlr])E([ptbdfsvzmnl])6ra$',
 u'Era-wajE': u'^(.*)vEra$',
 u'O.6ra-o.E': u'^(.*)O([ptbdfsvzmnlr])6ra$',
 u'Odra-alE': u'^(.*[fv])Odra$',
 u'Ora-avE': u'^sOra$',
 u'd6ra-': u'^sud6ra$',
 u'dra-lE': u'^vudra$',
 u'dra-zE': u'^(.*)kudra$',
 u'ira-E': u'^(.*[ptkbdgfsSvzZmnJNjrwHiyEe926auOo\xea\xfb\xe2\xf4][fvjrwHiyEe926auOo\xea\xfb\xe2\xf4][ptkbdgfsSvzZmnJNjr])ira$',
 u'ira-alE': u'^ira$',
 u'ira-jE': u'^(.*[ptkbdgfsSvzZmnJNlr])ira$',
 u'jEra-EjE': u'^(.*)sjEra$',
 u'j\xeadra-6nE': u'^(.*[ptbdfsvz])j\xeadra$',
 u'ora-avE': u'^ora$',
 u'r-sE': u'^sErvir$',
 u'ra-E': u'^(.*[ptkbdgfsSvzZmnJNjlrwHE96O\xea\xfb\xf4][mnJNjlrwHiyEe926auOo\xea\xfb\xe2\xf4][ptkbdgfsSvzZjrE6])ra$',
 u'ra-HE': u'^(.*)yra$',
 u'ra-jE': u'^(.*[ptkbdgfsSvzZmnJN